In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from dotenv import load_dotenv
import os
import requests
import pickle
load_dotenv("./.env")

False

In [6]:
df = pd.read_csv("./../data/labelled_resume_data.csv")
df.sample()

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
2584,NaN,NaN,"['C#', 'ASP.NET', 'Visual Basic', 'Ms Visual S...","['University of Texas Arlington', 'Federal Uni...","['Master of Science', 'Bachelor of Engineering']","['December 2014', 'June 2004']","['4.00', 'N/A']","['GPA', 'N/A']","['Industrial and Manufacturing Engineering', '...","['Company Name', 'Company Name', 'Company Name...",...,[None],['July 2013'],['N/A'],Executive/ Sr. Executive -IT,Bachelor of Science (BSc) in Computer Science ...,3 to 5 years,Age at most 40 years,Hardware & Software Installation\nSystem Monit...,NaN,0.716667


In [7]:
#function to select the top two entities from a list and make separate dataframe columns

# Date parsing functions
def parse_date(date_str):
    try:
        if pd.isna(date_str):
            return pd.NaT
        if isinstance(date_str, list):
            date_str = date_str[0]
        if isinstance(date_str, str) and date_str.strip() in ['Present', 'Till Date']:
            return pd.Timestamp.today()
            
        # Try specific formats first
        for fmt in ['%b %Y', '%B %Y', '%Y-%m-%d', '%Y/%m/%d', '%d-%m-%Y', '%d/%m/%Y']:
            try:
                return pd.to_datetime(date_str, format=fmt)
            except (ValueError, TypeError):
                continue
                
        # Fall back to pandas default parser
        return pd.to_datetime(date_str)
    except Exception as e:
        print(f"Error parsing date '{date_str}': {str(e)}")
        return pd.NaT
    
def get_top_two_entities(df: pd.DataFrame, column: str) -> pd.DataFrame:
    # Create new columns first
    df[f"{column}_1"] = None
    df[f"{column}_2"] = None
    
    for i, row in df.iterrows():
        entities = str(row[column]).split(',')  # Convert to string and split
        entities = [e.strip() for e in entities]  # Strip whitespace
        
        if len(entities) > 1:
            df.at[i, f"{column}_1"] = entities[0]  # Using .at is more efficient for single value
            df.at[i, f"{column}_2"] = entities[1]
        else:
            df.at[i, f"{column}_1"] = entities[0]
            df.at[i, f"{column}_2"] = None
    
    return df

def get_top_four_entities(df: pd.DataFrame, column: str) -> pd.DataFrame:
    # Create new columns first
    df[f"{column}_1"] = None
    df[f"{column}_2"] = None
    df[f"{column}_3"] = None
    df[f"{column}_4"] = None
    
    for i, row in df.iterrows():
        entities = str(row[column]).split(',')  # Convert to string and split
        entities = [e.strip() for e in entities]  # Strip whitespace
        
        if len(entities) > 3:
            df.at[i, f"{column}_1"] = entities[0]
            df.at[i, f"{column}_2"] = entities[1]
            df.at[i, f"{column}_3"] = entities[2]
            df.at[i, f"{column}_4"] = entities[3]
        elif len(entities) == 3:
            df.at[i, f"{column}_1"] = entities[0]
            df.at[i, f"{column}_2"] = entities[1]
            df.at[i, f"{column}_3"] = entities[2]
            df.at[i, f"{column}_4"] = None
        elif len(entities) == 2:
            df.at[i, f"{column}_1"] = entities[0]
            df.at[i, f"{column}_2"] = entities[1]
            df.at[i, f"{column}_3"] = None
            df.at[i, f"{column}_4"] = None
        else:
            df.at[i, f"{column}_1"] = entities[0]
            df.at[i, f"{column}_2"] = None
            df.at[i, f"{column}_3"] = None
            df.at[i, f"{column}_4"] = None
    
    return df
def tf_idf_vectorizer(df: pd.DataFrame, column: str):    
    # Initialize vectorizer
    vectorizer = TfidfVectorizer()
    
    # Fit and transform the column
    try:
        vectors = vectorizer.fit_transform(df[column].fillna(''))
        # Convert to dense array for easier handling
        dense_vectors = vectors.todense()
        
        # Replace column with vectorized version
        df[column] = list(dense_vectors)
        with open("./../data/{column}_vectorizer.pkl", "wb") as f:
            pickle.dump(vectorizer, f)
        return df
    except Exception as e:
        print(f"Error vectorizing {column}: {str(e)}")
        return df

#function to get embeddings
def get_response(data):
    api_key = os.getenv("JINA_API_KEY")
    URL = "https://api.jina.ai/v1/embeddings"

    resp = requests.post(URL, headers={"Authorization": f"Bearer {api_key}"}, json={"input": data, "model" : "jina-embeddings-v3", "task" : "classification"})
    return resp.json()["data"][0]["embedding"]

def get_embeddings(df, columns):
    for i, row in df.iterrows():
        for column in columns:
            try:
                if pd.notna(row[column]):
                    df.at[i, column] = get_response(str(row[column]))
                if i % 10 == 0:  # Print progress every 10 rows
                    print(f"Processing row {i}")
            except Exception as e:
                print(f"Error processing {column} at row {i}: {str(e)}")
                df.at[i, column] = None
    return df

def parse_experience_req(col, df):

    #Read the first number encountered in the string
    df[col] = df[col].str.extract(r'(\d+)').astype(float)

def preprocess_certain_cols(df, column, na_value = "MISSING"):
    
    df[column] = df[column].fillna(na_value)
    return df

In [8]:
from ast import parse


columns_to_ignore = ["address",'passing_years', 'educational_results', 'result_types', 'major_field_of_studies','professional_company_names', 'company_urls', 'related_skils_in_job', 'positions','locations','extra_curricular_activity_types','extra_curricular_organization_names','extra_curricular_organization_links', 'languages','proficiency_levels', 'certification_providers', 'certification_skills','online_links', 'issue_dates', 'expiry_dates', "educational_institution_name", "role_positions","degree_names", "career_objective", "start_dates", "end_dates", "age_requirement"]

#Add date_time
#For startdates
# df['start_dates'] = df['start_dates'].apply(lambda x : parse_date(x[0]) if isinstance(x, (list, tuple)) else x)


#For enddates
# def till_date_parsing(date):
#     if date == 'Till Date' or date == 'Present':
#         return pd.to_datetime('today')
#     else:
#         return date
# df.end_dates = df.end_dates.apply(till_date_parsing)
# # Date processing
# df['end_dates'] = df['end_dates'].apply(lambda x : parse_date(x[0]) if isinstance(x, (list, tuple)) else x)

#Modify columns in dataframe

#Add experience duration
# df['experience_duration'] = df.end_dates - df.start_dates

#create two columns for top two entities
get_top_two_entities(df, "educational_institution_name")
get_top_two_entities(df, "role_positions")
get_top_four_entities(df, "degree_names")

#rename_columns
rename_cols = {
    "educationaL_requirements" : "educational_requirements",
    'experiencere_requirement' : 'experience_requirement',
    'responsibilities.1' : 'job_responsibilities',
    "﻿job_position_name" : "job_position_name",
}
df.rename(columns=rename_cols, inplace=True)
#vectorize small features
#conconate all the entities in the list to a single string
df['skills'] = df['skills'].apply(lambda x: ' '.join(x) if isinstance(x, (list, tuple)) else str(x))
tf_idf_vectorizer(df, "skills")
df['skills_required'] = df['skills_required'].apply(lambda x: ' '.join(x).split('\n') if isinstance(x, (list, tuple)) else str(x))
tf_idf_vectorizer(df, "skills_required")
df['responsibilities'] = df['responsibilities'].apply(lambda x: ' '.join(x).split('\n') if isinstance(x, (list, tuple)) else str(x))
df["job_responsibilities"] = df["job_responsibilities"].apply(lambda x: ' '.join(x).split('\n') if isinstance(x, (list, tuple)) else str(x))
tf_idf_vectorizer(df, "responsibilities")
tf_idf_vectorizer(df, "job_position_name")
#parse experience requirement
parse_experience_req("experience_requirement", df)

#drop columns
df = df.drop(columns=columns_to_ignore, errors='ignore')

#Fill missing vals in educational_req
df.educational_requirements = df.educational_requirements.fillna(df.educational_requirements.mode()[0])
df.experience_requirement = df.experience_requirement.fillna(df.experience_requirement.median())

#Preprocess certain columns
df = preprocess_certain_cols(df, "job_responsibilities")
df = preprocess_certain_cols(df, "educational_institution_name_2")
df = preprocess_certain_cols(df, "role_positions_2")
df = preprocess_certain_cols(df, "degree_names_2")
df = preprocess_certain_cols(df, "degree_names_3")
df = preprocess_certain_cols(df, "degree_names_4")

#tf idf again
tf_idf_vectorizer(df, "job_responsibilities")
tf_idf_vectorizer(df, "educational_requirements")
tf_idf_vectorizer(df, "educational_institution_name_1")
tf_idf_vectorizer(df, "educational_institution_name_2")
tf_idf_vectorizer(df, "role_positions_1")
tf_idf_vectorizer(df, "role_positions_2")
tf_idf_vectorizer(df, "degree_names_1")
tf_idf_vectorizer(df, "degree_names_2")
tf_idf_vectorizer(df, "degree_names_3")
tf_idf_vectorizer(df, "degree_names_4")

,skills,responsibilities,job_position_name,educational_requirements,experience_requirement,job_responsibilities,skills_required,matched_score,educational_institution_name_1,educational_institution_name_2,role_positions_1,role_positions_2,degree_names_1,degree_names_2,degree_names_3,degree_names_4
0,[[[[[0. 0. 0. ... 0. 0. 0.]]]]],[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,1.0,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,0.850000,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....
1,[[[[[0. 0. 0. ... 0. 0. 0.]]]]],[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0.285165...,5.0,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,0.750000,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....
2,[[[[[0. 0. 0. ... 0. 0. 0.]]]]],[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0.43813845 0. 0. ...,3.0,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0.319322...,0.416667,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....
3,[[[[[0. 0. 0. ... 0. 0. 0.]]]]],[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,1.0,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0.36023539 0. ...,0.760000,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....
4,[[[[[0. 0. 0. ... 0. 0. 0.]]]]],[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,4.0,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,0.650000,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,[[[[[0. 0. 0. ... 0. 0. 0.]]]]],[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,5.0,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,0.683333,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....
9540,[[[[[0. 0. 0. ... 0. 0. 0.]]]]],[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. ...,3.0,[[[[[0. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,0.650000,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Assume df contains sparse vectors in the given columns
sparse_cols = ["skills", "responsibilities", "job_position_name", 
               "educational_requirements", "job_responsibilities", 
               "skills_required", "educational_institution_name_1",
               "educational_institution_name_2", "role_positions_1", 
               "role_positions_2", "degree_names_1", "degree_names_2", 
               "degree_names_3", "degree_names_4"]

import numpy as np

sparse_cols = ["skills", "responsibilities", "job_position_name", "educational_requirements", 
               "job_responsibilities", "skills_required", "educational_institution_name_1",
               "educational_institution_name_2", "role_positions_1", "role_positions_2",
               "degree_names_1", "degree_names_2", "degree_names_3", "degree_names_4"]

# Convert sparse vector columns into proper NumPy arrays
X_sparse = np.hstack([np.stack(df[col].to_numpy()) for col in sparse_cols])  # Ensures proper 2D structure

# Extract non-embedding features
X_other = df.drop(columns=["matched_score"] + sparse_cols).to_numpy(dtype = np.float32)  # Converts remaining features to NumPy

# Combine all features
X = np.hstack((X_other, X_sparse))  # Final feature set

# Target variable
y = df["matched_score"].to_numpy()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define MLP Model
class MLPModel(nn.Module):
    def __init__(self, input_size):
        super(MLPModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()  # Ensures output is between 0 and 1
        )

    def forward(self, x):
        return self.model(x)

# Instantiate model
input_size = X_train.shape[1]  # Includes embeddings + other features
model = MLPModel(input_size)

# Loss and optimizer
criterion = nn.MSELoss()  # Regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    print("Predictions:", y_pred[:5])


/home/bhaskar/miniconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([7635])) that is different to the input size (torch.Size([7635, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 10/10, Loss: 0.0385
Predictions: tensor([[0.5759],
        [0.5689],
        [0.5562],
        [0.5752],
        [0.5735]])


In [25]:
import pickle
with open("./models/model.pkl", "wb") as f:
    pickle.dump(model, f)

In [22]:
#evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
# Calculate MSE and R^2
mse = mean_squared_error(y_test, y_pred.numpy())
r2 = r2_score(y_test, y_pred.numpy())
print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

Mean Squared Error: 0.0276
R^2 Score: 0.0005


In [ ]:
import pickle
#save the dataframe
with open("./data/processed_data.pkl", "wb") as f:
    pickle.dump(df, f)

print("Data processing done")

Data processing done


skills                            9544
responsibilities                  9544
job_position_name                 9544
educational_requirements          9544
experience_requirement            9544
job_responsibilities              9544
skills_required                   9544
matched_score                     9544
educational_institution_name_1    9544
educational_institution_name_2    9544
role_positions_1                  9544
role_positions_2                  9544
degree_names_1                    9544
degree_names_2                    9544
degree_names_3                    9544
degree_names_4                    9544
dtype: int64

In [ ]:
#Build a ml model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

X,y  = df.drop(columns=["matched_score"]), df["matched_score"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
input_feature_size = len(X_train.columns)
X = np.array(X.tolist())  # Converts lists to a proper NumPy array
mlp_model = MLPRegressor(
    hidden_layer_sizes = (input_feature_size, 50, 10, 1),
    activation = 'relu',
    solver = 'adam',
    max_iter = 20,
    alpha=0.0001,
    batch_size='auto', 
    learning_rate="constant",
    learning_rate_init=0.001,
    tol=1e-4
)
mlp_model.fit(X_train, y_train)


ValueError: setting an array element with a sequence.